In [3]:
import warnings
warnings.filterwarnings("ignore")

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [4]:
import pandas as pd

from molfeat.trans import MoleculeTransformer

from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_decomposition import PLSRegression

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score

from sklearn.pipeline import Pipeline

from qsarcons.consensus import RandomSearchRegressor, SystematicSearchRegressor, GeneticSearchRegressor

### 1. Load dataset

Load training and test set for QSAR model building and evaluation

In [5]:
data_train = pd.read_csv("benchmark_collection_original/molnet/lipophilicity/train.csv", header=None)
data_test = pd.read_csv("benchmark_collection_original/molnet/lipophilicity/test.csv", header=None)
#
smiles_train, smiles_test = data_train[0].to_list(), data_test[0].to_list()
y_train, y_test = data_train[1].to_list(), data_test[1].to_list()

### 2. Define a list of chemical descriptors

In this example, the molfeat package is used to readily calculate chemical descriptors

In [6]:
descr_dict = {
    'scaffoldkeys': MoleculeTransformer(featurizer='scaffoldkeys', dtype=float),
    'secfp': MoleculeTransformer(featurizer='secfp', dtype=float),
    'atompair-count': MoleculeTransformer(featurizer='atompair-count', dtype=float),
    'avalon': MoleculeTransformer(featurizer='avalon', dtype=float),
    'ecfp-count': MoleculeTransformer(featurizer='ecfp-count', dtype=float),
    'ecfp': MoleculeTransformer(featurizer='ecfp', dtype=float),
    'erg': MoleculeTransformer(featurizer='erg', dtype=float),
    'estate': MoleculeTransformer(featurizer='estate', dtype=float),
    'fcfp-count': MoleculeTransformer(featurizer='fcfp-count', dtype=float),
    'fcfp': MoleculeTransformer(featurizer='fcfp', dtype=float),
    'maccs': MoleculeTransformer(featurizer='maccs', dtype=float),
    'pattern': MoleculeTransformer(featurizer='pattern', dtype=float),
    'rdkit': MoleculeTransformer(featurizer='rdkit', dtype=float),
    'topological-count': MoleculeTransformer(featurizer='topological-count', dtype=float),
    'topological': MoleculeTransformer(featurizer='topological', dtype=float),
    'layered': MoleculeTransformer(featurizer='layered', dtype=float),
    'erg': MoleculeTransformer(featurizer='erg', dtype=float),
    'desc2D': MoleculeTransformer(featurizer='desc2D', dtype=float)
}

### 3. Define a list of machine learning methods.

In this example, scikit-learn package is used to build machine learning models.

In [7]:
ml_dict = {
            'XGBRegressor': XGBRegressor(),
            'PLSRegression': PLSRegression(),
            'RidgeRegression': Ridge(),
            # 'LinearRegression': LinearRegression(),
            'RandomForestRegressor': RandomForestRegressor(),
            'MLPRegressor': MLPRegressor(max_iter=1000, solver='adam'),
            'SVR': SVR(),
            'KNeighborsRegressor': KNeighborsRegressor()
}

### 4. Build QSAR models

Build a QSAR model for each pair of descriptors and machine learning method. 

In [12]:
train_pred_df = pd.DataFrame()
test_pred_df = pd.DataFrame()
for descr_name, descr_func in descr_dict.items():

    # calc descriptors
    x_train, x_test = descr_func(smiles_train), descr_func(smiles_test)
    
    # scale descriptors
    scaler = MinMaxScaler()
    scaler.fit(x_train)
    x_train_scaled, x_test_scaled = scaler.transform(x_train), scaler.transform(x_test)
    
    for ml_name, ml_func in ml_dict.items():

        # build QSAR model and make training set predictions
        ml_func.fit(x_train_scaled, y_train)
        y_train_pred = ml_func.predict(x_train_scaled)
        y_test_pred = ml_func.predict(x_test_scaled)

        # save training set predictions
        model_name = f"{descr_name}|{ml_name}"
        
        train_pred_df[model_name] = y_train_pred
        test_pred_df[model_name] = y_test_pred

In [13]:
test_pred_df

,scaffoldkeys|XGBRegressor,scaffoldkeys|PLSRegression,scaffoldkeys|RidgeRegression,scaffoldkeys|RandomForestRegressor,scaffoldkeys|MLPRegressor,scaffoldkeys|SVR,scaffoldkeys|KNeighborsRegressor,secfp|XGBRegressor,secfp|PLSRegression,secfp|RidgeRegression,...,topological|SVR,topological|KNeighborsRegressor,layered|XGBRegressor,layered|PLSRegression,layered|RidgeRegression,layered|RandomForestRegressor,layered|MLPRegressor,layered|SVR,layered|KNeighborsRegressor,desc2D|XGBRegressor
0,2.369648,2.173712,2.393000,2.653067,2.683378,2.658775,2.282,3.083099,3.604889,3.629538,...,3.179646,2.670,2.952314,2.520175,3.210509,2.595400,2.102910,2.386845,2.654,3.336913
1,1.639252,1.854237,1.985207,1.595713,1.118863,1.741977,0.762,1.997536,1.751864,2.014809,...,1.619913,2.136,1.571251,1.708380,1.498257,1.602048,1.745835,1.641866,1.374,1.691936
2,1.364088,2.104582,2.522786,2.455447,2.975915,2.831791,2.708,2.481389,0.370460,3.058709,...,0.832713,0.046,0.160766,1.525969,1.269704,0.964433,1.444367,0.917468,-0.030,0.724250
3,1.790467,1.475200,1.686352,1.902650,1.990494,2.254512,2.536,2.329862,2.223149,2.422689,...,2.165340,2.126,1.967909,1.911787,5.241532,2.374125,2.127350,2.347779,1.652,1.088050
4,2.810759,0.780561,1.081520,2.439100,2.236514,1.197422,2.182,2.544699,2.480240,2.274215,...,2.238990,2.520,2.379151,1.615969,2.269790,2.420500,2.364355,2.265164,2.522,1.866123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,3.073426,1.974345,1.588275,2.758050,3.051895,2.829684,3.188,1.757544,1.921530,1.636126,...,1.859021,1.586,2.161216,2.378488,2.920998,2.494000,2.536307,2.598211,1.992,2.635026
836,3.049445,0.605623,0.665272,1.928000,1.381862,0.487015,0.516,1.210042,0.851962,1.647185,...,1.059020,0.586,0.656173,0.582671,1.953048,1.116200,0.490598,0.833632,0.322,2.159050
837,2.789806,2.161075,2.171393,2.124557,2.185152,2.272564,1.662,1.768473,1.968762,1.988315,...,2.484249,2.154,2.286902,1.324777,2.801140,1.652400,2.356881,1.476263,2.216,2.827075
838,2.996418,1.700135,1.818382,2.950058,3.113008,2.946275,3.062,2.768366,3.410359,2.254444,...,3.206302,3.086,3.003054,2.329786,4.765581,3.167043,3.314234,3.093426,3.176,2.554442


### 5. Find optimal consensus of QSAR models

QSAR model predictions for the training set are used to find the optimal consensus of models of fixed size. Then, the optimal consensus is used to generate consensus predictions for the test set

In [14]:
cons_size = 10

cons_dict = {
             "BestModel":SystematicSearchRegressor(cons_size=1, metric='r2'),
             "AllMean": SystematicSearchRegressor(cons_size=10**10, metric='r2'),
             "RandomConsensus":RandomSearchRegressor(cons_size=cons_size, n_iter=10000, metric='r2'),
             "SystematicConsensus":SystematicSearchRegressor(cons_size=cons_size, metric='r2'),
             "GeneticConsensus": GeneticSearchRegressor(cons_size=cons_size, mut_prob=0.2, metric='r2')
            }

In [15]:
cons_pred_df = pd.DataFrame()
for cons_name, cons_func in cons_dict.items():

    # find optimal consensus based on training set predictions
    best_cons = cons_func.run(train_pred_df, y_train)

    # make consensus test set predictions
    y_pred = test_pred_df[best_cons].mean(axis=1)
    cons_pred_df[cons_name] = y_pred

In [16]:
cons_pred_df

,BestModel,AllMean,RandomConsensus,SystematicConsensus,GeneticConsensus
0,3.666625,2.747319,2.230121,3.463121,3.344321
1,1.582488,1.656320,1.560283,1.649485,1.655667
2,2.032143,0.914113,1.230779,0.867688,0.651115
3,2.466228,1.971181,1.954695,1.944438,2.026857
4,2.500365,2.361810,2.406588,2.557187,2.669151
...,...,...,...,...,...
835,1.660744,2.103634,2.446472,1.845927,1.864122
836,1.151411,1.441131,1.262763,1.805537,1.831999
837,1.442705,2.177206,1.985895,2.649618,2.601317
838,2.848445,2.979345,3.017534,2.978896,3.057128


Compare the final accuracy of test set predictions for consensus search methods

In [17]:
for cons_name in cons_pred_df:
    r2_test = r2_score(y_test, cons_pred_df[cons_name])
    print(f"{cons_name}: {r2_test:.2f}")

BestModel: 0.55
AllMean: 0.64
RandomConsensus: 0.59
SystematicConsensus: 0.71
GeneticConsensus: 0.71
